In [12]:
# hocemo da iskoristimo SVN iz sklearn da razdvojimo podatke iz iris skupa

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# SVC je za classifier, a SVR je za regressor, ovde radimo klasifikaciju pa koristimo prvi
from sklearn.svm import SVC

In [2]:
df = pd.read_csv('../iris.csv')
df.head()

,Sepal_Length,Sepal_Width,Petal_Length,Petal_Width,Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [3]:
X = df.drop('Species', axis=1)
y = df['Species']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

In [5]:
print(X_train.shape)
print(X_test.shape)

(105, 4)
(45, 4)


In [6]:
# pogodno nam je da ovde izvrsimo normalizaciju ili standardizaciju (zasto (?))
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler()

In [7]:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [8]:
# ostavljamo default parametre, ali vidimo da mozemo da podesimo c, tj tezine koje dodajemo greskama 
model = SVC()

In [9]:
model.fit(X_train, y_train)

SVC()

In [10]:
y_predicted = model.predict(X_test)
print(accuracy_score(y_test, y_predicted))

0.9333333333333333


In [16]:
# mozemo npr da probamo sa grid search da trazimo optimalne parametre, samo moramo da pazimo na sledece
# ako bismo napravili smao jednu mapu sa parametrima kao u sledecm primeru, nece se lepo racunati za neke modele
# npr ako je jedan od parametara kernel i moguca vrednost 'linear', ako mi u mapi imamo i neke vrednosti
#  za parametar gamma, one ce biti ignorisane, zato sto linearni kernel ne moze da ima gama parametar

# bolji pristup je da napravimo listu mapa, gde cemo davati moguce parametre u svakoj mapi koji 
# imaju smisla u tom kontekstu (naravno i prvi pristup ce da radi, ali trajace vise vremena)
# C treba da bude neki broj izmedju 0 (malo vrednujemo gresku) i 1 (puno vrednujemo gresku)
params = [  {
             'kernel' : ['linear'],
             'C' : [pow(2, x) for x in range(-5, 5)]
            },
            {
              'kernel' : ['poly'],
              'degree' : [2, 3, 4, 5],
              'C' : [pow(2, x) for x in range(-5, 5)],
              'gamma' : np.arange(0, 1, 0.1)
            }
         ]

In [17]:
model = GridSearchCV(SVC(), param_grid=params, cv=5)

In [18]:
model.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid=[{'C': [0.03125, 0.0625, 0.125, 0.25, 0.5, 1, 2, 4, 8,
                                16],
                          'kernel': ['linear']},
                         {'C': [0.03125, 0.0625, 0.125, 0.25, 0.5, 1, 2, 4, 8,
                                16],
                          'degree': [2, 3, 4, 5],
                          'gamma': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]),
                          'kernel': ['poly']}])

In [22]:
# vidimo sta se ispotavilo najbolje
print(model.best_estimator_)

SVC(C=0.125, kernel='linear')


In [27]:
# vidimo koliko ima potpornih vektora za svaku klasu (nisam skapirao bas sta su tacno ti vektori)
print(model.best_estimator_.n_support_)

[ 6 26 20]


In [28]:
# vidimo tacno i koji su to vektori
print(model.best_estimator_.support_vectors_)

[[-0.85542696  0.56565306 -1.15815801 -0.91130239]
 [-1.56640187 -1.6422864  -1.38663446 -1.17599603]
 [-1.21091441 -0.09672878 -1.32951534 -1.17599603]
 [-1.21091441  0.12406517 -1.21527712 -1.30834285]
 [-0.14445205  1.6696228  -1.15815801 -1.17599603]
 [-0.97392278 -0.09672878 -1.21527712 -1.30834285]
 [ 0.44802705 -0.31752272  0.32693892  0.14747217]
 [-0.14445205 -0.97990456 -0.13001398 -0.24956829]
 [ 1.04050614  0.12406517  0.38405803  0.27981899]
 [ 0.56652287  0.56565306  0.55541537  0.54451263]
 [ 0.92201032 -0.31752272  0.49829626  0.14747217]
 [ 0.09253959  0.34485912  0.61253448  0.80920627]
 [ 1.15900196 -0.53831667  0.61253448  0.27981899]
 [ 0.44802705 -1.86308035  0.44117715  0.41216581]
 [-0.26294786 -0.09672878  0.44117715  0.41216581]
 [ 0.21103541 -0.31752272  0.44117715  0.41216581]
 [ 1.27749778  0.12406517  0.6696536   0.41216581]
 [ 0.32953123 -0.09672878  0.49829626  0.27981899]
 [ 1.3959936   0.34485912  0.55541537  0.27981899]
 [ 1.04050614  0.12406517  0.55

In [30]:
# mozda nam onaj ispis iznad ne znaci mnogo, pa mozemod a vidimo i tacno indekse tih instanci koje cine
# odredjen potporni vektor 
print(model.best_estimator_.support_)

[ 20  37  70  73  79  80   2   4   7  16  17  23  24  30  33  35  36  41
  42  44  52  57  58  67  75  83  84  86  91  94 101 102   0   5   9  15
  27  28  34  43  45  50  51  60  63  64  66  71  72  85  93  99]


In [31]:
y_predicted = model.predict(X_test)
print(accuracy_score(y_test, y_predicted))

0.9555555555555556
